# Rail Fence šifras

##Šifravimas
Šifro principas: raidžių išdėstymas zigzagu ir nuskaitymas į vieną žodį eilutė po eilutės.

Pavyzdžiui: Žodis "WE ARE DISCOVERED FLEE AT ONCE" Sudėjimas zigzagu, kur kas atitinkamaą intervalą yra keičiama teksto rašymo kryptis (aukštyn - žemyn). Šiuo atveju - kas 3.

    W . . . . D I . . . . E R . . . . E E . . . . C E
    . E . . E . . S . . V . . E . . L . . A . . N . .
    . . A R . . . . C O . . . . D F . . . . T O . . .
    
Užšifruojant tekstą nuskaitoma pirmoji, antroji, ..., n-toji eilutė (priklausomai nuo parinkto intervalo užšifruojant, šiuo atveju - 3):

 1-oji eilutė - `W D I E R E E C E`
 
 2-oji eilutė - `E E S V E L A N`
 
 3-oji eilutė - `A R C O D F T O`

ir paeiliui visos eilutės sudedamos į vieną:

Užšifruotas tekstas - `WDIEREECEEESVELANARCODFTO`


In [9]:
def encode(plain_text, key):
    cipher_text = ""
    #išimami tarpai
    plain_text = plain_text.replace(" ", "")
        
    for line_index in xrange(0, key):
        letter_index = 0
        cipher_text += plain_text[line_index]
        
        while letter_index + 2 * key - (line_index + 1) < len(plain_text):
            #Egzistuoja poros, kurių viršūnės susiliečia
            letter_index += 2 * key
            cipher_text += plain_text[letter_index - (line_index + 1)] 
            
            if letter_index + line_index >= len(plain_text):
                break
            cipher_text += plain_text[letter_index + line_index]

    return cipher_text

In [10]:
raktas = 3
tekstas = "WE ARE DISCOVERED FLEE AT ONCE"
encode(tekstas, raktas)

'WDIEREECEEESVELANARCODFTO'

##Iššifravimas

Iššifravimui reikalingas užšifruotas tekstas ir raktas - intervalas kada buvo keičiama kryptis.
Visas principas yra pagrįstas atvirkštiniu būdu, nei buvo užšifruota:
 1. Sužinomos eilutės
 2. Zigzagu atšifruojamas tekstas

Pirmasis žingsnis turi šiokių tokių svarbių aspektų:
 1. Gali egzistuoti "uodega" - paskutinis stulpelis, kuris nėra užpildytas;
 2. Uodega svarbi tuo, kad ji gali būti užrašoma tiek iš viršaus, tiek iš apačios - reikia nustatyti kryptį ir raidžių uodegoje kiekį;
 3. Žinant kiekį, reikia nustatyti kaip nuskaityti užšifruotą tekstą (jei nuo viršaus rašomos uodegos n raidžių, tai pirmosios n eilučių bus ilgesnės vienu simboliu ir atvirkščiai - iš apačios rašomos uodegos n raidžių, tai paskutinės n eilučių bus vieni simboliu ilgesnės).

In [15]:
import math

def decode(cipher_text, key):
    #Reikia sužinoti ar yra uodega ir kuriame gale
    #uodega iš viršaus į apačią - teigiama
    #iš apačios į viršų - neigiama
    #
    #pradžioje pradedama nuo viršaus į apačią, nelyginis skaičius
    plain_text = ""
    columns = int(math.ceil(len(cipher_text) / float(key)))
    tail = len(cipher_text) % key #nustatomas ilgis
    tail *= -1 if columns % 2 == 0 else 1 #nustatoma kryptis
    ind = 0 #zingsnis, kurio pagalba kopijuojamas tekstas į grupes
    grupes = [] #grupes, tai atbuliniu zingsniu sudaryta tvorele
    for i in xrange(0, key):
        #pridedamas papildomas kiekis (1 daugiau) simbolių, nei įprastai
        if (tail > 0 and i <= tail - 1) or (tail < 0 and i >= key + tail):
            grupes.append(cipher_text[ind: ind + columns])
            ind += columns
        else:
            grupes.append(cipher_text[ind: ind + columns - 1])
            ind += columns - 1

    #gautos grupes, formuojamas tekstas
    for column in xrange(0, columns):
        iterator = xrange(0, key)
        
        if column % 2 == 1:
            #apverciamas iterator
            iterator = reversed(iterator)

        for eilute in iterator:
            if (column <= len(grupes[eilute]) - 1):
                plain_text += grupes[eilute][column]

    return plain_text

raktas


In [19]:
decode("WDIEREECEEESVELANARCODFTO", 3)


'WEAREDISCOVEREDFLEEATONCE'

##Kriptoanalizė

Realaus sprendinio, kur tiksliai sužinomas pradinis tekstas - nėra. Galimi spėjimai keičiant iššifruojant su vis kitokiu rakto dydžiu (raktų variacija nėra didesnė už teksto ilgį)

In [33]:
import sys
sys.path.insert(0, '../L1')
from Ciphers import RailFence

In [40]:
rf = RailFence()
P = "WE ARE DISCOVERED FLEE AT ONCE"
K = 4
C = rf.encode(P, K)
P = P.replace(" ", "")
P_array = rf.cryptoanalyze(C)
P in P_array.values()

True